In [1]:
from llm.llm.chatgpt import ChatGPT
from llm.llm.gemini import Gemini
from setup_db import create_chroma_db, connect_to_db, execute_query, DBHUB
from branch_reasoning import *

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = ChatGPT()

In [3]:
db_name = 'test_db'
user = 'postgres'
password = '12345678'
port = '5433'
host = 'localhost'

In [4]:
conn = {'db_name' : 'test_db',
'user' : 'postgres',
'password' : '12345678',
'port' : '5433',
'host' : 'localhost'}

In [5]:
# conn = connect_to_db(db_name, user, password, host, port)

In [ ]:
collection_chromadb = 'category_bank_chroma'
persist_directory = 'data/category_bank_chroma'
bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_non_bank_chroma'
persist_directory = 'data/category_non_bank_chroma'
none_bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_sec_chroma'
persist_directory = 'data/category_sec_chroma'
sec_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_ratio_chroma'
persist_directory = 'data/category_ratio_chroma'
ratio_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'company_name_chroma'
persist_directory = 'data/company_name_chroma'
vector_db_company = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'sql_query'
persist_directory = 'data/sql_query'
vector_db_sql = create_chroma_db(collection_chromadb, persist_directory)

In [ ]:
db = DBHUB(conn, bank_vector_store, none_bank_vector_store, sec_vector_store, ratio_vector_store, vector_db_company, vector_db_sql)

In [ ]:
prompt =  "Get ROA, ROE of all the company which are owned by VinGroup"

In [ ]:
history, error_messages, execution_tables = llm_branch_reasoning(llm, prompt, db, verbose=True)

CompletionUsage(completion_tokens=100, prompt_tokens=381, total_tokens=481, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0), prompt_tokens_details={'cached_tokens': 0})
Branch reasoning response: 
```json
{
    "steps": [
        "Step 1: Retrieve the list of companies owned by VinGroup from the `sub_and_shareholder` database.",
        "Step 2: Extract the financial data (net income and total assets) for each company from the appropriate financial report database (`bank_financial_report` or `non_bank_financial_report`).",
        "Step 3: Calculate ROA and ROE for each company using the extracted financial data."
    ]
}
```
CompletionUsage(completion_tokens=40, prompt_tokens=165, total_tokens=205, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0), prompt_tokens_details={'cached_tokens': 0})
Find suitable column response: 
```json
{
    "bank_column_name": [],
    "non_bank_column_name": [
        "Net Income",
        "Total Assets",
        

In [12]:
for message in history:
    print("==============")
    print(message['role'])
    print('_____________')
    print(message['content'])

system
_____________
You are an expert in financial statement and database management. You will be asked to convert a natural language query into a PostgreSQL query.
user
_____________
You have the following database schema:

<description>
You're given database about the financial reports of top Vietnamese companies, both bank and cooperates.

All the data in the financial report are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translation of the categories are followed by the International Financial Reporting Standards (IFRS).



Here are the descriptions of tables in the database:



### Table: `bank_financial_report`

This table store information of financial report for bank



|stock_code|year|quarter|category_code|data|

|---|----|----|----|----|----|

|VCB|2023|  0 | BS_300 | 1839613.198 |

|LPB|2024|  2 | CF_045 | 68522.835|

|BID|2024|  1 | IS_014 | 5392.606 |



The table has six columns:

 - `stock_code`: The trading symbol of the bank.

 - 